In [1]:
%load_ext autoreload
%autoreload 2

import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

import torch
from torch.autograd import Variable
import os
from metal.mmtl.cxr.analysis.notebooks.cxr_analysis import load_log_json, load_results_from_log
from metal.mmtl.cxr.cxr_tasks import create_tasks_and_payloads

In [2]:
# Setting env variables
os.environ['CXRDATA']='/data/datasets/mmtl/cxr'
os.environ['CXR8IMAGES']='/data/datasets/nih/images/images'

In [3]:
# Loading results from model log
model_log = '/home/jdunnmon/Research/repos/metal/logs/2019_05_07/test_naive_new_code_19_21_28'
log_jsons = load_results_from_log(model_log)

In [13]:
task_config = log_jsons['task_config']
task_config['subsample'] = -1
task_config['eval_finding'] = 'ALL'
task_names=['CXR8-DRAIN_ALL']
task_config['sample_dict']={"train_sample_dict":{'PNEUMOTHORAX':-1,'ALL':300}}
                            #"valid_sample_dict":{'PNEUMOTHORAX':-1,'ALL':300}}
task_config['add_normal_col']=True
task_config['slice_dict']={"CXR8-DRAIN_PNEUMOTHORAX": ["chest_drain_cnn_neg"]}
task_config['slice_pos_only']=['NONE']
task_config['active_slice_heads']={'pred':True,'ind':False}
task_config['verbose']=True
#task_config['train_sampler'] = "imbalanced_mmtl_sampler"
tasks, payloads = create_tasks_and_payloads(task_names, **task_config, batch_size=16)

metal.utils - INFO - Could not find kwarg "shuffle" in destination dict.
metal.utils - INFO - Could not find kwarg "attention" in destination dict.
metal.utils - INFO - Could not find kwarg "verbose" in destination dict.
metal.mmtl.cxr.cxr_tasks - INFO - Loading CXR8-DRAIN Dataset
metal.mmtl.cxr.cxr_tasks - INFO - Using train finding ALL
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_datasets - INFO - Using pre-specified sampling dictionary
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_datasets - INFO - Adding normal column to dataset
metal.mmtl.cxr.cxr_tasks - INFO - Shuffling training split...
metal.mmtl.payload - INFO - Added label_set with 4525/7388 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred to payload CXR8-DRAIN_train.
metal.mmtl.payload - INFO - Added label_set with 10923/11259 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred to payload CXR8-DRAIN_v

In [14]:
# Checking to make sure our slice prediction task is reasonably balanced
from collections import Counter
print(Counter(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred']))

Counter({2: 10715, 0: 336, 1: 208})


In [15]:
# Getting slice prediction task only
task_names = [t.name for t in tasks]
slice_prediction_task = tasks[task_names.index('CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred')]

In [16]:
from metal.mmtl.metal_model import MetalModel
# Initializing the model
model_config = log_jsons['model_config'] 
model = MetalModel(tasks=tasks, **model_config)
missing, unexpected = model.load_weights(os.path.join('/home/jdunnmon/Research/repos/metal/logs/2019_05_07/test_naive_new_code_19_21_28','best_model.pth'))

metal.mmtl.metal_model - INFO - Your destination state dict has different keys for the update key.
metal.mmtl.metal_model - INFO - Loading without strictness.


In [48]:
unexpected

[]

In [17]:
# Training on dev!
import copy
train_on_dev = False
if train_on_dev:
    payloads[0] = copy.deepcopy(payloads[1])
    payloads[0].name = 'CXR8-DRAIN_train'
    payloads[0].split = 'train'

In [37]:
trainer_config

{'verbose': True,
 'seed': '1701',
 'commit_hash': None,
 'ami': None,
 'progress_bar': False,
 'n_epochs': 20,
 'l2': 0.0,
 'grad_clip': 1.0,
 'optimizer_config': {'optimizer': 'adam',
  'optimizer_common': {'lr': 0.0001},
  'sgd_config': {'momentum': 0.9},
  'adam_config': {'betas': [0.9, 0.999]},
  'rmsprop_config': {}},
 'lr_scheduler': 'linear',
 'lr_scheduler_config': {'warmup_steps': 0.0,
  'warmup_unit': 'epochs',
  'min_lr': 1e-05,
  'exponential_config': {'gamma': 0.999},
  'plateau_config': {'factor': 0.5, 'patience': 10, 'threshold': 0.0001}},
 'metrics_config': {'task_metrics': [],
  'trainer_metrics': ['model/valid/all/loss'],
  'aggregate_metric_fns': [],
  'max_valid_examples': 0,
  'valid_split': 'valid',
  'test_split': 'test'},
 'task_scheduler': 'proportional',
 'freeze': None,
 'logger': True,
 'logger_config': {'log_unit': 'epochs',
  'log_every': 1.0,
  'score_every': 1.0,
  'log_lr': True},
 'train_schedule_plan': {'plan': {'-1': ['CXR8-DRAIN_ATELECTASIS',
    '

In [44]:
# Setting up trainer
from metal.mmtl.trainer import MultitaskTrainer
trainer_config = log_jsons['config'] 

tasks_to_freeze = [t.name for t in tasks if t.name != slice_prediction_task.name]
trainer_config["train_schedule_plan"] = {
                "plan": {
                    "-1": tasks_to_freeze,
                    },
                "freeze": "heads",
                }
trainer_config['n_epochs']=50
trainer_config['checkpoint_config']['checkpoint_metric'] = 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1'
trainer_config['checkpoint_config']['checkpoint_dir']='checkpoints_temp'
trainer_config['checkpoint_config']['checkpoint_metric_mode'] = 'max'
trainer_config['writer_config']['run_name'] = 'test_slice_train_balanced_train_on_train'

# Initializing trainer 
trainer = MultitaskTrainer(**trainer_config)

In [45]:
tasks_to_freeze

['CXR8-DRAIN_ATELECTASIS',
 'CXR8-DRAIN_CARDIOMEGALY',
 'CXR8-DRAIN_EFFUSION',
 'CXR8-DRAIN_INFILTRATION',
 'CXR8-DRAIN_MASS',
 'CXR8-DRAIN_NODULE',
 'CXR8-DRAIN_PNEUMONIA',
 'CXR8-DRAIN_PNEUMOTHORAX',
 'CXR8-DRAIN_CONSOLIDATION',
 'CXR8-DRAIN_EDEMA',
 'CXR8-DRAIN_EMPHYSEMA',
 'CXR8-DRAIN_FIBROSIS',
 'CXR8-DRAIN_PLEURAL_THICKENING',
 'CXR8-DRAIN_HERNIA']

In [46]:
# Training model
trainer.train_model(model, payloads, train_schedule_plan=trainer_config['train_schedule_plan'])

metal.mmtl.trainer - INFO - Training on training set...
metal.mmtl.trainer - INFO - Beginning train loop.
metal.mmtl.trainer - INFO - Expecting a total of approximately 7392 examples and 462 batches per epoch from 1 payload(s) in the train split.
metal.mmtl.trainer - INFO - Initializing new log writer...
/home/jdunnmon/Research/repos/metal/metal/mmtl/trainer.py:645: UserWarning: You have provided checkpoint_dir, overriding the default of using log_dir/run_dir/run_name/checkpoints. Be careful: multiple concurrent runs may override each other.
  warnings.warn(msg)
metal.logging.writer - INFO - Writing config to /home/jdunnmon/Research/repos/metal/logs/2019_05_13/test_slice_train_balanced_train_on_train_01_18_43/config.json
metal.mmtl.trainer - INFO - Freezing ['CXR8-DRAIN_ATELECTASIS', 'CXR8-DRAIN_CARDIOMEGALY', 'CXR8-DRAIN_EFFUSION', 'CXR8-DRAIN_INFILTRATION', 'CXR8-DRAIN_MASS', 'CXR8-DRAIN_NODULE', 'CXR8-DRAIN_PNEUMONIA', 'CXR8-DRAIN_PNEUMOTHORAX', 'CXR8-DRAIN_CONSOLIDATION', 'CXR8-DRA

[1.0 epo]: CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred:[CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss=8.66e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1=7.28e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc=6.67e-01, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy=8.14e-01] model:[train/all/loss=8.66e-02, train/all/lr=9.80e-05, valid/all/loss=3.80e-01] CXR8-DRAIN_ATELECTASIS:[CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/roc-auc=5.73e-01, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/accuracy=9.01e-01] CXR8-DRAIN_CARDIOMEGALY:[CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/roc-auc=5.05e-01, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/accuracy=9.78e-01] CXR8-DRAIN_EFFUSION:[CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/roc-auc=6.54e-01, CXR

metal.mmtl.trainer - INFO - Freezing ['CXR8-DRAIN_ATELECTASIS', 'CXR8-DRAIN_CARDIOMEGALY', 'CXR8-DRAIN_EFFUSION', 'CXR8-DRAIN_INFILTRATION', 'CXR8-DRAIN_MASS', 'CXR8-DRAIN_NODULE', 'CXR8-DRAIN_PNEUMONIA', 'CXR8-DRAIN_PNEUMOTHORAX', 'CXR8-DRAIN_CONSOLIDATION', 'CXR8-DRAIN_EDEMA', 'CXR8-DRAIN_EMPHYSEMA', 'CXR8-DRAIN_FIBROSIS', 'CXR8-DRAIN_PLEURAL_THICKENING', 'CXR8-DRAIN_HERNIA'] heads
metal.mmtl.trainer - INFO - Training tasks {'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred'}
root - INFO - Summary name CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss is illegal; using CXR8-DRAIN_PNEUMOTHORAX_slice_chest_drain_cnn_neg_pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice_chest_drain_cnn_neg_pred/loss instead.
root - INFO - Summary name CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_

[2.0 epo]: CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred:[CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss=8.69e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/f1=9.30e-02, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/roc-auc=6.94e-01, CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/accuracy=9.36e-01] model:[train/all/loss=8.69e-02, train/all/lr=9.60e-05, valid/all/loss=2.96e-01] CXR8-DRAIN_ATELECTASIS:[CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/roc-auc=5.76e-01, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/accuracy=9.01e-01] CXR8-DRAIN_CARDIOMEGALY:[CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/roc-auc=5.40e-01, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/accuracy=9.78e-01] CXR8-DRAIN_EFFUSION:[CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/roc-auc=6.30e-01, CXR

metal.mmtl.trainer - INFO - Freezing ['CXR8-DRAIN_ATELECTASIS', 'CXR8-DRAIN_CARDIOMEGALY', 'CXR8-DRAIN_EFFUSION', 'CXR8-DRAIN_INFILTRATION', 'CXR8-DRAIN_MASS', 'CXR8-DRAIN_NODULE', 'CXR8-DRAIN_PNEUMONIA', 'CXR8-DRAIN_PNEUMOTHORAX', 'CXR8-DRAIN_CONSOLIDATION', 'CXR8-DRAIN_EDEMA', 'CXR8-DRAIN_EMPHYSEMA', 'CXR8-DRAIN_FIBROSIS', 'CXR8-DRAIN_PLEURAL_THICKENING', 'CXR8-DRAIN_HERNIA'] heads
metal.mmtl.trainer - INFO - Training tasks {'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred'}
root - INFO - Summary name CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/loss is illegal; using CXR8-DRAIN_PNEUMOTHORAX_slice_chest_drain_cnn_neg_pred/CXR8-DRAIN_train/CXR8-DRAIN_PNEUMOTHORAX_slice_chest_drain_cnn_neg_pred/loss instead.
root - INFO - Summary name CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred/CXR8-DRAIN_valid/CXR8-DRAIN_PNEUMOTHORAX_slice:chest_

...

[50.00 epo]: model:[train/all/lr=1.00e-05, valid/all/loss=3.11e-01] CXR8-DRAIN_ATELECTASIS:[CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/roc-auc=6.20e-01, CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/accuracy=9.01e-01] CXR8-DRAIN_CARDIOMEGALY:[CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/roc-auc=6.82e-01, CXR8-DRAIN_valid/CXR8-DRAIN_CARDIOMEGALY/accuracy=9.78e-01] CXR8-DRAIN_EFFUSION:[CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/roc-auc=7.16e-01, CXR8-DRAIN_valid/CXR8-DRAIN_EFFUSION/accuracy=8.84e-01] CXR8-DRAIN_INFILTRATION:[CXR8-DRAIN_valid/CXR8-DRAIN_INFILTRATION/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_INFILTRATION/roc-auc=6.15e-01, CXR8-DRAIN_valid/CXR8-DRAIN_INFILTRATION/accuracy=8.21e-01] CXR8-DRAIN_MASS:[CXR8-DRAIN_valid/CXR8-DRAIN_MASS/f1=0, CXR8-DRAIN_valid/CXR8-DRAIN_MASS/roc-auc=6.74e-01, CXR8-DRAIN_valid/CXR8-DRAIN_MASS/accuracy=9.43e-01] CXR8-DRAIN_NODULE:[CXR8-DRAIN

metal.mmtl.trainer - INFO - Finished training
/home/jdunnmon/Research/repos/metal/metal/mmtl/trainer.py:1009: UserWarning: MeTaL does not support calculating loss on the test set.
  warnings.warn(msg)
metal.mmtl.trainer - INFO - Cleaning checkpoints


{'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/accuracy': 0.8916625731255303,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/f1': 0,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/roc-auc': 0.6677299954240242,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_train/CXR8-DRAIN_ATELECTASIS/accuracy': 0.8226854358419058,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_train/CXR8-DRAIN_ATELECTASIS/f1': 0,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_train/CXR8-DRAIN_ATELECTASIS/roc-auc': 0.5917420857101949,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/accuracy': 0.9011457500666133,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/f1': 0,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_valid/CXR8-DRAIN_ATELECTASIS/roc-auc': 0.6478883812952634,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_test/CXR8-DRAIN_CARDIOMEGALY/accuracy': 0.9743669896842764,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_test/CXR8-DRAIN_CARDIOMEGALY/f1': 0,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_tes

metal.logging.writer - INFO - Writing metrics to /home/jdunnmon/Research/repos/metal/logs/2019_05_13/test_slice_train_balanced_train_on_train_01_18_43/metrics.json
metal.logging.writer - INFO - Writing log to /home/jdunnmon/Research/repos/metal/logs/2019_05_13/test_slice_train_balanced_train_on_train_01_18_43/log.json
metal.mmtl.trainer - INFO - Full model saved at /home/jdunnmon/Research/repos/metal/logs/2019_05_13/test_slice_train_balanced_train_on_train_01_18_43/model.pkl


{'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/f1': 0,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/roc-auc': 0.6677299954240242,
 'CXR8-DRAIN_ATELECTASIS/CXR8-DRAIN_test/CXR8-DRAIN_ATELECTASIS/accuracy': 0.8916625731255303,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_test/CXR8-DRAIN_CARDIOMEGALY/f1': 0,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_test/CXR8-DRAIN_CARDIOMEGALY/roc-auc': 0.6177654716432455,
 'CXR8-DRAIN_CARDIOMEGALY/CXR8-DRAIN_test/CXR8-DRAIN_CARDIOMEGALY/accuracy': 0.9743669896842764,
 'CXR8-DRAIN_EFFUSION/CXR8-DRAIN_test/CXR8-DRAIN_EFFUSION/f1': 0,
 'CXR8-DRAIN_EFFUSION/CXR8-DRAIN_test/CXR8-DRAIN_EFFUSION/roc-auc': 0.742728286235765,
 'CXR8-DRAIN_EFFUSION/CXR8-DRAIN_test/CXR8-DRAIN_EFFUSION/accuracy': 0.8778636180949404,
 'CXR8-DRAIN_INFILTRATION/CXR8-DRAIN_test/CXR8-DRAIN_INFILTRATION/f1': 0,
 'CXR8-DRAIN_INFILTRATION/CXR8-DRAIN_test/CXR8-DRAIN_INFILTRATION/roc-auc': 0.6143688305747648,
 'CXR8-DRAIN_INFILTRATION/CXR8-DRAIN_test/CXR8-DRAIN_INFILTR

## SANDBOX

In [272]:
payloads[0].labels_to_tasks

{'CXR8-DRAIN_ATELECTASIS': 'CXR8-DRAIN_ATELECTASIS',
 'CXR8-DRAIN_CARDIOMEGALY': 'CXR8-DRAIN_CARDIOMEGALY',
 'CXR8-DRAIN_EFFUSION': 'CXR8-DRAIN_EFFUSION',
 'CXR8-DRAIN_INFILTRATION': 'CXR8-DRAIN_INFILTRATION',
 'CXR8-DRAIN_MASS': 'CXR8-DRAIN_MASS',
 'CXR8-DRAIN_NODULE': 'CXR8-DRAIN_NODULE',
 'CXR8-DRAIN_PNEUMONIA': 'CXR8-DRAIN_PNEUMONIA',
 'CXR8-DRAIN_PNEUMOTHORAX': 'CXR8-DRAIN_PNEUMOTHORAX',
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred',
 'CXR8-DRAIN_CONSOLIDATION': 'CXR8-DRAIN_CONSOLIDATION',
 'CXR8-DRAIN_EDEMA': 'CXR8-DRAIN_EDEMA',
 'CXR8-DRAIN_EMPHYSEMA': 'CXR8-DRAIN_EMPHYSEMA',
 'CXR8-DRAIN_FIBROSIS': 'CXR8-DRAIN_FIBROSIS',
 'CXR8-DRAIN_PLEURAL_THICKENING': 'CXR8-DRAIN_PLEURAL_THICKENING',
 'CXR8-DRAIN_HERNIA': 'CXR8-DRAIN_HERNIA'}

In [243]:
slice_prediction_task

ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred, loss_multiplier=1.00)

In [278]:
import torch
#np.unique(payloads[0].data_loader.sampler.weights,return_counts=True)

AttributeError: 'SequentialSampler' object has no attribute 'weights'

In [276]:
payloads[1].data_loader.dataset.df.sum()

FOLLOW-UP #                                                       11370
PATIENT ID                                                     12032637
PATIENT AGE           038Y042Y038Y058Y041Y065Y065Y019Y054Y057Y048Y05...
PATIENT GENDER        FMMMMMMFMMMMFMMMFMMMFFMMMMFFMFMMFMMMFFFFFMFMMF...
VIEW POSITION         PAAPPAAPAPAPAPPAAPAPAPPAAPPAPAPAPAAPAPPAAPPAAP...
CARDIOMEGALY                                                         35
EMPHYSEMA                                                            95
EFFUSION                                                            180
HERNIA                                                               20
INFILTRATION                                                        169
MASS                                                                 87
NODULE                                                               80
ATELECTASIS                                                         130
PNEUMOTHORAX                                                    

In [280]:
import numpy as np
np.sum(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred']==2)

262

In [181]:
import numpy as np
np.sum(payloads[1].data_loader.dataset.labels['CXR8-DRAIN_PNEUMOTHORAX']==1)

544

In [203]:
payloads[1].data_loader.dataset.labels

{'CXR8-DRAIN_ATELECTASIS': array([1, 1, 1, ..., 2, 2, 2]),
 'CXR8-DRAIN_CARDIOMEGALY': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EFFUSION': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_INFILTRATION': array([1, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_MASS': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_NODULE': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PNEUMONIA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PNEUMOTHORAX': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_CONSOLIDATION': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EDEMA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_EMPHYSEMA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_FIBROSIS': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_PLEURAL_THICKENING': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_HERNIA': array([2, 2, 2, ..., 2, 2, 2]),
 'CXR8-DRAIN_NORMAL': array([2, 2, 2, ..., 1, 1, 1]),
 'CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred': array([0, 0, 0, ..., 0, 0, 0])}

In [238]:
tasks

[ClassificationTask(name=CXR8-DRAIN_ATELECTASIS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_CARDIOMEGALY, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EFFUSION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_INFILTRATION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_MASS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_NODULE, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMONIA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_CONSOLIDATION, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EDEMA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_EMPHYSEMA, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_FIBROSIS, loss_multiplier=1.00),
 ClassificationTask(name=CXR8-DRAIN_PLEURAL_THICKENING,